In [105]:
!pip install datasets Pillow einops torch torchvision python-dotenv timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.9 MB/s eta 0:00:00a 0:00:01


In [95]:
from pathlib import Path

# Specify the token
hf_token = 'your_huggingface_token'

# Define the path to the .env file
env_path = Path('.env')

# Write the token to the .env file
with env_path.open('w') as env_file:
    env_file.write(f'HF_TOKEN={hf_token}\n')

print(f".env file created at {env_path.resolve()}")

.env file created at /home/jovyan/work/.env


In [96]:
import requests
import os
from dotenv import load_dotenv

def download_files(token, urls, output_paths):
    """
    Downloads files from the specified URLs and saves them to the given output paths.

    :param token: Hugging Face authorization token
    :param urls: List of URLs to download
    :param output_paths: List of paths to save the downloaded files
    """
    headers = {"Authorization": f"Bearer {token}"}

    for url, output_path in zip(urls, output_paths):
        response = requests.get(url, headers=headers, stream=True)
        if response.status_code == 200:
            with open(output_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            print(f"Download of {url} completed and saved to {output_path}!")
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")

# Load the .env file
load_dotenv()

# Retrieve the token
token = os.getenv('HF_TOKEN')

urls = [
    "https://huggingface.co/datasets/ILSVRC/imagenet-1k/resolve/main/data/val_images.tar.gz?download=true",
    "https://huggingface.co/datasets/ILSVRC/imagenet-1k/resolve/main/classes.py?download=true"
]
output_paths = ["val_images.tar.gz", "classes.py"]

download_files(token, urls, output_paths)

Download of https://huggingface.co/datasets/ILSVRC/imagenet-1k/resolve/main/data/val_images.tar.gz?download=true completed and saved to val_images.tar.gz!
Download of https://huggingface.co/datasets/ILSVRC/imagenet-1k/resolve/main/classes.py?download=true completed and saved to classes.py!


In [97]:
import classes
import os
import tarfile
import shutil
from collections import OrderedDict

# Extract the tar.gz file
def extract_tar_gz(file_path, extract_path):
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    with tarfile.open(file_path, 'r:gz') as tar:
        tar.extractall(path=extract_path)
    print(f"Extracted {file_path} to {extract_path}")

# Create directories for each class and move the images
def organize_images(data_path):
    for filename in os.listdir(data_path):
        if filename.endswith(".JPEG"):
            parts = filename.split('_')
            if len(parts) > 1:
                # Reconstruct the filename without the class ID
                new_filename = '_'.join(parts[:-1]) + ".JPEG"
                old_filepath = os.path.join(data_path, filename)
                new_filepath = os.path.join(data_path, new_filename)
                os.rename(old_filepath, new_filepath)

# Paths and dictionary from classes.py
val_images_path = "val_images.tar.gz"
extract_path = "val_images"

# Import classes.py
IMAGENET2012_CLASSES = classes.IMAGENET2012_CLASSES

# Execute the functions
extract_tar_gz(val_images_path, extract_path)
organize_images(extract_path)

Extracted val_images.tar.gz to val_images


In [98]:
!git clone https://github.com/liuchongming1999/ImageNet-1K-training-and-validation.git

Cloning into 'ImageNet-1K-training-and-validation'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 49 (delta 19), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (49/49), 361.11 KiB | 7.08 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [99]:
!mv ./ImageNet-1K-training-and-validation/valprep.sh val_images

In [100]:
%cd val_images
!bash ./valprep.sh
%cd ..

/home/jovyan/work/val_images
/home/jovyan/work


In [101]:
!mv val_images/valprep.sh .

In [102]:
!git clone https://github.com/NX-AI/vision-lstm.git

Cloning into 'vision-lstm'...
remote: Enumerating objects: 613, done.
remote: Counting objects: 100% (613/613), done.
remote: Compressing objects: 100% (438/438), done.
remote: Total 613 (delta 189), reused 574 (delta 150), pack-reused 0
Receiving objects: 100% (613/613), 888.34 KiB | 9.87 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [107]:
!python vision-lstm/eval.py --data val_images --name vil-base

using device: cuda:0
initializing ImageNet-1K validation set 'val_images'
loading model 'vil-base'
Using cache found in /home/jovyan/.cache/torch/hub/nx-ai_vision-lstm_main
batch_size: 32
num_workers: 10
100%|███████████████████████████████████████| 1563/1563 [02:33<00:00, 10.16it/s]
accuracy: 81.63%
